# Data Load


In [45]:
import numpy as np
import pandas as pd
import math

from scipy import stats #Analysis 
from scipy.stats import norm 

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

In [46]:
# 한글 폰트가 깨지면 실행하세요! 
import matplotlib.font_manager as fm
fontlist = fm.findSystemFonts(fontpaths = None, fontext='ttf')
# 아래 주석을 지워서 폰트 리스트를 확인하고 한글 폰트를 font_path에 추가합니다
fontlist[:]

from matplotlib import font_manager, rc
#font_path = 'C:\\Users\\mtang\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumSquare.ttf'
font_path = 'C:\\WINDOWS\\Fonts\\NanumGothicLight.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [47]:
# pandas option 설정 하는 코드
# monthly_gain의 경우 부동소수점으로 나타나서 보기 어려울땐 윗 줄의 주석을 제거하고 아래에 주석을 추가하고
# 다시 원래대로 돌리고 싶다면 아래에 주석제거, 위 코드에 주석추가
pd.options.display.float_format = '{:.2f}'.format
#pd.reset_option('display.float_format')

In [48]:
path = "./Dataset/"

data = pd.read_csv(path+"kwproja_data_location.csv")

data

,DATE_,SHOP_CODE,SHOP_NAME,SHOP_TYPE_BIG,SHOP_TYPE_SMALL,LONGITUDE,LATITUDE,MONTHLY_GAIN,AVERAGE_SALE_PRICE,GU,DONG
0,201709,9381000145,고기.원칙,한식,고기구이/육회 전문점,127.12,37.49,19439629.62,31288,송파구,가락본동
1,201709,18211290214,동달식당 강남본점,한식,고기구이/육회 전문점,127.02,37.50,184992017.85,62762,서초구,서초2동
2,201708,14946450207,서귀포혹돈,한식,고기구이/육회 전문점,126.90,37.47,217826541.22,89076,금천구,독산1동
3,201709,6188500699,(주)아이언미트 숙달돼지 사당점,한식,고기구이/육회 전문점,126.98,37.48,52677315.50,87160,서초구,방배2동
4,201708,5598500495,민소한우(서린점),한식,고기구이/육회 전문점,126.98,37.57,75889137.73,93559,종로구,종로1.2.3.4가동
...,...,...,...,...,...,...,...,...,...,...,...
2927734,201701,4609200378,일신수산,수산물,회/수산물 일반,126.94,37.51,33788752.81,93138,동작구,노량진2동
2927735,201611,57372070107,흑산홍어,수산물,회/수산물 일반,126.90,37.54,3615713.59,20000,영등포구,양평2동
2927736,201701,1631700892,해초섬 수릿골,수산물,회/수산물 일반,127.06,37.51,55862791.29,69776,강남구,삼성2동
2927737,201611,11760800468,명불허전 원조골뱅이 목동사거리점,수산물,회/수산물 일반,126.86,37.53,19247620.58,44923,강서구,화곡4동


# 2. EDA 

2,927,739 rows X 11 columns

- 매장 속성 정보
  - shop_code -> 식별자 feature => drop   
  - shop_name => DL
  - longtitude : 경도, latitude : 위도 -> 매장 위치 (회사 근처, 학교 근처 등 매출 영향성 있음) -> K-mean clustering => geo, ML, DL
  - address1, address2 : GeoEncoder를 통해 따로 얻은 행정동, 1(30), 2(436) => DL
  - shop_type_big -> 15 category, shop_type_big_label, ML => DL
  - shop_type_small -> 61 category, shop_type_small_label, ML => DL 

- 매출 정보
  - date -> 24 category, 201606~ 201805 까지의 data
  - monthly_gain / avearge_sale_price = 한달 총 판매수

- 매출 통계 정보-> X

In [49]:
# rename data columns and check the data
data.columns = ['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small', 
                'longitude', 'latitude', 'monthly_gain', 'average_sale_price', 'address1', 'address2']

print(data.columns, '\n')
print(data.info())

Index(['date', 'shop_code', 'shop_name', 'shop_type_big', 'shop_type_small',
       'longitude', 'latitude', 'monthly_gain', 'average_sale_price',
       'address1', 'address2'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927739 entries, 0 to 2927738
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                int64  
 1   shop_code           int64  
 2   shop_name           object 
 3   shop_type_big       object 
 4   shop_type_small     object 
 5   longitude           float64
 6   latitude            float64
 7   monthly_gain        float64
 8   average_sale_price  int64  
 9   address1            object 
 10  address2            object 
dtypes: float64(3), int64(3), object(5)
memory usage: 245.7+ MB
None


# 3. Preprocessing 
- df_geo : 구(address1)를 기준으로 통계량을 붙여놓은 데이터프레임 
    - monthly_gain + average_sale_price + shop_type_big 개수 
    - count : 데이터 개수 
    - mg : monthly_gain 약자 
    - sp : average_sale_price 약자 
    - mean, st, ~ max : 평균값, 표준편차, ~ 최대값 
    - shop_type_big(15) : 각 업종대분류에 해당하는 데이터가 해당 지역(구)에는 몇 개가 있는가? 
    - 0, 1, 2 : 각 구마다 1등, 2등, 3등으로 많은 업종 대분류는? 
    - **27 rows × 34 columns**
- df_shop : 업종 대분류(shop_type_big)을 기준으로 통계량을 붙여놓은 데이터프레임 
    - **15 rows × 49 columns**
- 구 -> label encoder => (30개)
    - '강남구' '강동구' '강북구' '강서구' '고양시 덕양구' '과천시' '관악구' '광명시' '광진구' '구로구' '구리시'
      '금천구' '노원구' '도봉구' '동대문구' '동작구' '마포구' '서대문구' '서초구' '성동구' '성북구' '송파구' '양천구'
      '영등포구' '용산구' '은평구' '종로구' '중구' '중랑구' '하남시' 

In [50]:
# 원본 data와 따로 관리 -> original data = data, preprocessed data = processed_data 
processed_data = data.copy()
processed_data

,date,shop_code,shop_name,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,address1,address2
0,201709,9381000145,고기.원칙,한식,고기구이/육회 전문점,127.12,37.49,19439629.62,31288,송파구,가락본동
1,201709,18211290214,동달식당 강남본점,한식,고기구이/육회 전문점,127.02,37.50,184992017.85,62762,서초구,서초2동
2,201708,14946450207,서귀포혹돈,한식,고기구이/육회 전문점,126.90,37.47,217826541.22,89076,금천구,독산1동
3,201709,6188500699,(주)아이언미트 숙달돼지 사당점,한식,고기구이/육회 전문점,126.98,37.48,52677315.50,87160,서초구,방배2동
4,201708,5598500495,민소한우(서린점),한식,고기구이/육회 전문점,126.98,37.57,75889137.73,93559,종로구,종로1.2.3.4가동
...,...,...,...,...,...,...,...,...,...,...,...
2927734,201701,4609200378,일신수산,수산물,회/수산물 일반,126.94,37.51,33788752.81,93138,동작구,노량진2동
2927735,201611,57372070107,흑산홍어,수산물,회/수산물 일반,126.90,37.54,3615713.59,20000,영등포구,양평2동
2927736,201701,1631700892,해초섬 수릿골,수산물,회/수산물 일반,127.06,37.51,55862791.29,69776,강남구,삼성2동
2927737,201611,11760800468,명불허전 원조골뱅이 목동사거리점,수산물,회/수산물 일반,126.86,37.53,19247620.58,44923,강서구,화곡4동


### geo

In [28]:
# monthly_gain 
group1 = processed_data[['monthly_gain', 'address1']].groupby('address1')
df1 = group1.describe().droplevel(axis=1, level=0)

# round
df1.iloc[:, 1:] = df1.iloc[:, 1:].applymap(lambda x : round(x, -4))
df1.columns = ['count', 'mean_mg', 'st_mg', 'min_mg', '25%_mg', '50%_mg', '75%_mg', 'max_mg']

# sum - 총 매출
df1['monthly_gain'] = group1.sum().apply(lambda x : round(x, -4))

df1.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain
address1,,,,,,,,,
강남구,256562.00,52570000.00,808920000.00,0.00,9140000.00,21480000.00,44510000.00,91386200000.00,13486576890000.00
강동구,112134.00,23380000.00,172690000.00,0.00,3330000.00,8670000.00,20740000.00,10642020000.00,2621232850000.00
강북구,84071.00,18260000.00,122340000.00,0.00,2530000.00,6770000.00,17730000.00,9668020000.00,1534726850000.00
강서구,142272.00,34730000.00,357880000.00,0.00,4600000.00,12250000.00,29090000.00,23156200000.00,4941651280000.00
과천시,61.00,13960000.00,7970000.00,1340000.00,6190000.00,13440000.00,21100000.00,32190000.00,851370000.00


In [29]:
# average_sale_price
df2 = processed_data[['average_sale_price', 'address1']].groupby('address1')
df2 = df2.describe().droplevel(axis=1, level=0)

# round
df2.iloc[:, 1:] = df2.iloc[:, 1:].applymap(lambda x : round(x, -4))
df2.columns = ['count', 'mean_sp', 'st_sp', 'min_sp', '25%_sp', '50%_sp', '75%_sp', 'max_sp']
df2.head()

,count,mean_sp,st_sp,min_sp,25%_sp,50%_sp,75%_sp,max_sp
address1,,,,,,,,
강남구,256562.00,60000.00,320000.00,0.00,10000.00,20000.00,50000.00,50000000.00
강동구,112134.00,30000.00,140000.00,0.00,10000.00,20000.00,40000.00,17000000.00
강북구,84071.00,30000.00,90000.00,0.00,10000.00,20000.00,40000.00,10180000.00
강서구,142272.00,40000.00,120000.00,0.00,10000.00,20000.00,40000.00,7040000.00
과천시,61.00,30000.00,10000.00,10000.00,10000.00,20000.00,40000.00,50000.00


In [30]:
# shop_type_big
df3 = processed_data.groupby(['address1', 'shop_type_big']).size()
df3 = df3.unstack() # unstack : SQL 집계 결과를 가로, 세로 축으로 보기 좋게 나열 
df3 = df3.fillna(0) # NaN to 0
df3.head()

shop_type_big,다방/커피숍/카페,별식/퓨전요리,분식,뷔페,생활서비스,소매,수산물,양식,유흥주점,음식-미분류,일식,제과제빵떡케익,중식,패스트푸드,한식
address1,,,,,,,,,,,,,,,
강남구,32463.00,15488.00,14623.00,1524.00,0.00,29177.00,10846.00,1862.00,18621.00,47551.00,9873.00,7745.00,11426.00,11701.00,43662.00
강동구,10495.00,7458.00,8668.00,1152.00,0.00,15625.00,5414.00,285.00,10000.00,16628.00,2759.00,4141.00,3558.00,4809.00,21142.00
강북구,6917.00,6879.00,5571.00,861.00,0.00,10493.00,4422.00,130.00,8149.00,12582.00,1693.00,2489.00,2391.00,3331.00,18163.00
강서구,13003.00,10642.00,9763.00,1068.00,0.00,19298.00,7605.00,440.00,12928.00,21673.00,3356.00,5665.00,4910.00,6603.00,25318.00
과천시,0.00,24.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.00,24.00


In [31]:
# rank for shop_type_big
df4 = df3.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=3)
df4.head()

,0,1,2
address1,,,
강남구,음식-미분류,한식,다방/커피숍/카페
강동구,한식,음식-미분류,소매
강북구,한식,음식-미분류,소매
강서구,한식,음식-미분류,소매
과천시,별식/퓨전요리,한식,패스트푸드


In [32]:
df_geo = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
df_geo = pd.concat([df_geo, df3], axis=1)
df_geo = pd.concat([df_geo, df4], axis=1)
df_geo.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain,mean_sp,...,유흥주점,음식-미분류,일식,제과제빵떡케익,중식,패스트푸드,한식,0,1,2
address1,,,,,,,,,,,,,,,,,,,,,
강남구,256562.00,52570000.00,808920000.00,0.00,9140000.00,21480000.00,44510000.00,91386200000.00,13486576890000.00,60000.00,...,18621.00,47551.00,9873.00,7745.00,11426.00,11701.00,43662.00,음식-미분류,한식,다방/커피숍/카페
강동구,112134.00,23380000.00,172690000.00,0.00,3330000.00,8670000.00,20740000.00,10642020000.00,2621232850000.00,30000.00,...,10000.00,16628.00,2759.00,4141.00,3558.00,4809.00,21142.00,한식,음식-미분류,소매
강북구,84071.00,18260000.00,122340000.00,0.00,2530000.00,6770000.00,17730000.00,9668020000.00,1534726850000.00,30000.00,...,8149.00,12582.00,1693.00,2489.00,2391.00,3331.00,18163.00,한식,음식-미분류,소매
강서구,142272.00,34730000.00,357880000.00,0.00,4600000.00,12250000.00,29090000.00,23156200000.00,4941651280000.00,40000.00,...,12928.00,21673.00,3356.00,5665.00,4910.00,6603.00,25318.00,한식,음식-미분류,소매
과천시,61.00,13960000.00,7970000.00,1340000.00,6190000.00,13440000.00,21100000.00,32190000.00,851370000.00,30000.00,...,0.00,0.00,0.00,0.00,0.00,13.00,24.00,별식/퓨전요리,한식,패스트푸드


In [94]:
df_geo.to_csv('df_geo.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### geo - date

In [33]:
# date 
df5 = processed_data[['date', 'address1', 'monthly_gain']].groupby(['address1', 'date']).mean().round(-4)
df6 = processed_data[['date', 'address1', 'monthly_gain']].groupby(['address1', 'date']).count()

df5 = df5.unstack()
df5 = df5.rename(columns = {'monthly_gain' : 'mean'})
df6 = df6.unstack()
df6 = df6.rename(columns = {'monthly_gain' : 'count'})

df_geo_date = pd.concat([df5, df6], axis=1)
df_geo_date.head()

mean                                                  \
date          201606      201607      201608      201609      201610   
address1                                                               
강남구      52340000.00 56110000.00 55070000.00 52060000.00 53380000.00   
강동구      23840000.00 24710000.00 24610000.00 24460000.00 23390000.00   
강북구      17820000.00 18520000.00 18560000.00 18790000.00 17970000.00   
강서구      35080000.00 36480000.00 35610000.00 35400000.00 35420000.00   
과천시      14010000.00 16770000.00 13570000.00 11760000.00 16940000.00   

                                                                      ...  \
date          201611      201612      201701      201702      201703  ...   
address1                                                              ...   
강남구      51310000.00 59010000.00 55350000.00 47080000.00 54360000.00  ...   
강동구      22560000.00 24850000.00 23760000.00 20950000.00 23870000.00  ...   
강북구      17460000.00 19870000.00 18550000.00 16770000.00 18770000.00  ...   
강서구      34340000.00 39170000.00 35120000.00 31190000.00 36630000.00  ...   
과천시      18090000.00 14190000.00 15780000.00 13670000.00 14210000.00  ...   

          count                                                                 
date     201708 201709 201710 201711 201712 201801 201802 201803 201804 201805  
address1                                                                        
강남구       10799  10764  10767  10778  10742  10616  10619  10512  10470  10548  
강동구        4707   4690   4693   4649   4688   4645   4634   4533   4550   4616  
강북구        3514   3490   3512   3498   3502   3478   3469   3438   3446   3497  
강서구        6014   5997   6017   6013   6009   5944   5917   5832   5841   5892  
과천시           2      2      2      2      2      2      2      2      2      2  

[5 rows x 48 columns]

In [34]:
df_geo_date.to_csv('df_geo_date.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### shop type big

In [35]:
# monthly_gain 
group1 = processed_data[['monthly_gain', 'shop_type_big']].groupby('shop_type_big')
df1 = group1.describe().droplevel(axis=1, level=0)

df1.iloc[:, 1:] = df1.iloc[:, 1:].applymap(lambda x : round(x, -4))
df1.columns = ['count', 'mean_mg', 'st_mg', 'min_mg', '25%_mg', '50%_mg', '75%_mg', 'max_mg']

# sum - 총 매출
df1['monthly_gain'] = group1.sum().apply(lambda x : round(x, -4))

df1.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain
shop_type_big,,,,,,,,,
다방/커피숍/카페,318339.00,13630000.00,26300000.00,0.00,3250000.00,7510000.00,15910000.00,2454380000.00,4339878830000.00
별식/퓨전요리,213619.00,19720000.00,28710000.00,0.00,5770000.00,12060000.00,23830000.00,1535740000.00,4212465640000.00
분식,188263.00,15050000.00,19540000.00,0.00,3470000.00,8710000.00,19690000.00,594570000.00,2834027670000.00
뷔페,24891.00,36000000.00,70220000.00,0.00,6550000.00,15850000.00,36200000.00,1246630000.00,896103630000.00
생활서비스,47.00,4970000.00,3440000.00,30000.00,2920000.00,4190000.00,6760000.00,15390000.00,233660000.00


In [36]:
# average_sale_price
df2 = processed_data[['average_sale_price', 'shop_type_big']].groupby('shop_type_big')
df2 = df2.describe().droplevel(axis=1, level=0)

df2.iloc[:, 1:] = df2.iloc[:, 1:].applymap(lambda x : round(x, -4))
df2.columns = ['count', 'mean_sp', 'st_sp', 'min_sp', '25%_sp', '50%_sp', '75%_sp', 'max_sp']
df2.head()

,count,mean_sp,st_sp,min_sp,25%_sp,50%_sp,75%_sp,max_sp
shop_type_big,,,,,,,,
다방/커피숍/카페,318339.00,10000.00,80000.00,0.00,10000.00,10000.00,10000.00,17520000.00
별식/퓨전요리,213619.00,30000.00,120000.00,0.00,20000.00,20000.00,30000.00,14280000.00
분식,188263.00,20000.00,60000.00,0.00,10000.00,10000.00,20000.00,15000000.00
뷔페,24891.00,70000.00,210000.00,0.00,20000.00,40000.00,60000.00,7450000.00
생활서비스,47.00,40000.00,40000.00,10000.00,20000.00,20000.00,60000.00,170000.00


In [37]:
# address1 - geo
df3 = processed_data.groupby(['shop_type_big', 'address1']).size()
df3 = df3.unstack() # unstack : SQL 집계 결과를 가로, 세로 축으로 보기 좋게 나열 
df3 = df3.fillna(0) # NaN to 0
df3.head()

address1,강남구,강동구,강북구,강서구,과천시,관악구,광명시,광진구,구로구,금천구,...,성동구,성북구,송파구,양천구,영등포구,용산구,은평구,종로구,중구,중랑구
shop_type_big,,,,,,,,,,,,,,,,,,,,,
다방/커피숍/카페,32463.00,10495.00,6917.00,13003.00,0.00,11065.00,0.00,10241.00,9044.00,7242.00,...,9320.00,11244.00,19248.00,8896.00,16601.00,12106.00,8714.00,18048.00,14653.00,6768.00
별식/퓨전요리,15488.00,7458.00,6879.00,10642.00,24.00,11099.00,0.00,7125.00,8372.00,5261.00,...,6175.00,8063.00,12996.00,8336.00,10300.00,5814.00,8473.00,6776.00,7214.00,7850.00
분식,14623.00,8668.00,5571.00,9763.00,0.00,7644.00,0.00,6169.00,7725.00,5136.00,...,4689.00,6543.00,12187.00,6991.00,9566.00,5359.00,7133.00,8032.00,7433.00,6096.00
뷔페,1524.00,1152.00,861.00,1068.00,0.00,1604.00,0.00,685.00,1238.00,978.00,...,759.00,855.00,1647.00,766.00,1211.00,629.00,1036.00,939.00,1005.00,1194.00
생활서비스,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24.00,0.00,...,23.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [38]:
# rank for address1(geo)
df4 = df3.apply(lambda s, n: pd.Series(s.nlargest(n).index), axis=1, n=3)
df4.head()

,0,1,2
shop_type_big,,,
다방/커피숍/카페,강남구,마포구,서초구
별식/퓨전요리,강남구,송파구,마포구
분식,강남구,송파구,마포구
뷔페,송파구,관악구,강남구
생활서비스,구로구,성동구,강남구


In [39]:
df_type = pd.concat([df1, df2.iloc[:, 1:]], axis=1)
df_type = pd.concat([df_type, df3], axis=1)
df_type = pd.concat([df_type, df4], axis=1)
df_type.head()

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain,mean_sp,...,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,0,1,2
shop_type_big,,,,,,,,,,,,,,,,,,,,,
다방/커피숍/카페,318339.00,13630000.00,26300000.00,0.00,3250000.00,7510000.00,15910000.00,2454380000.00,4339878830000.00,10000.00,...,8896.00,16601.00,12106.00,8714.00,18048.00,14653.00,6768.00,강남구,마포구,서초구
별식/퓨전요리,213619.00,19720000.00,28710000.00,0.00,5770000.00,12060000.00,23830000.00,1535740000.00,4212465640000.00,30000.00,...,8336.00,10300.00,5814.00,8473.00,6776.00,7214.00,7850.00,강남구,송파구,마포구
분식,188263.00,15050000.00,19540000.00,0.00,3470000.00,8710000.00,19690000.00,594570000.00,2834027670000.00,20000.00,...,6991.00,9566.00,5359.00,7133.00,8032.00,7433.00,6096.00,강남구,송파구,마포구
뷔페,24891.00,36000000.00,70220000.00,0.00,6550000.00,15850000.00,36200000.00,1246630000.00,896103630000.00,70000.00,...,766.00,1211.00,629.00,1036.00,939.00,1005.00,1194.00,송파구,관악구,강남구
생활서비스,47.00,4970000.00,3440000.00,30000.00,2920000.00,4190000.00,6760000.00,15390000.00,233660000.00,40000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,구로구,성동구,강남구


In [18]:
df_type

,count,mean_mg,st_mg,min_mg,25%_mg,50%_mg,75%_mg,max_mg,monthly_gain,mean_sp,...,양천구,영등포구,용산구,은평구,종로구,중구,중랑구,0,1,2
shop_type_big,,,,,,,,,,,,,,,,,,,,,
다방/커피숍/카페,318339.00,13630000.00,26300000.00,0.00,3250000.00,7510000.00,15910000.00,2454380000.00,4339878830000.00,10000.00,...,8896.00,16601.00,12106.00,8714.00,18048.00,14653.00,6768.00,강남구,마포구,서초구
별식/퓨전요리,213619.00,19720000.00,28710000.00,0.00,5770000.00,12060000.00,23830000.00,1535740000.00,4212465640000.00,30000.00,...,8336.00,10300.00,5814.00,8473.00,6776.00,7214.00,7850.00,강남구,송파구,마포구
분식,188263.00,15050000.00,19540000.00,0.00,3470000.00,8710000.00,19690000.00,594570000.00,2834027670000.00,20000.00,...,6991.00,9566.00,5359.00,7133.00,8032.00,7433.00,6096.00,강남구,송파구,마포구
뷔페,24891.00,36000000.00,70220000.00,0.00,6550000.00,15850000.00,36200000.00,1246630000.00,896103630000.00,70000.00,...,766.00,1211.00,629.00,1036.00,939.00,1005.00,1194.00,송파구,관악구,강남구
생활서비스,47.00,4970000.00,3440000.00,30000.00,2920000.00,4190000.00,6760000.00,15390000.00,233660000.00,40000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,구로구,성동구,강남구
소매,359034.00,116400000.00,1569080000.00,0.00,7380000.00,27180000.00,47730000.00,181237340000.00,41790631240000.00,40000.00,...,13464.00,16985.00,9487.00,13462.00,7117.00,14789.00,12161.00,강남구,송파구,강서구
수산물,131323.00,28340000.00,33010000.00,0.00,8460000.00,18100000.00,36320000.00,703400000.00,3721432720000.00,60000.00,...,3756.00,8149.00,3330.00,4618.00,4958.00,5433.00,5283.00,강남구,영등포구,송파구
양식,13726.00,25500000.00,38190000.00,10000.00,6090000.00,13860000.00,28730000.00,550880000.00,349946020000.00,50000.00,...,432.00,655.00,807.00,308.00,989.00,459.00,293.00,강남구,마포구,서초구
유흥주점,254766.00,15800000.00,22750000.00,0.00,3690000.00,9030000.00,19230000.00,1289820000.00,4026153940000.00,50000.00,...,7714.00,12527.00,9559.00,8768.00,9901.00,8426.00,9978.00,마포구,강남구,송파구


### Total

In [19]:
df_total = processed_data[['monthly_gain', 'average_sale_price']].describe().transpose()

# round - except for min, count 
df_total.iloc[:, 1:3] = df_total.iloc[:, 1:3].applymap(lambda x : round(x,  -4))
df_total.iloc[:, 4:] = df_total.iloc[:, 4:].applymap(lambda x : round(x,  -4))

df_total

,count,mean,std,min,25%,50%,75%,max
monthly_gain,2927739.00,32770000.00,552030000.00,5.06,4310000.00,11580000.00,27580000.00,181237340000.00
average_sale_price,2927739.00,40000.00,180000.00,1.00,10000.00,20000.00,40000.00,165260000.00


In [20]:
sum_mg = processed_data['monthly_gain'].sum().round(-4)
sum_sp = processed_data['average_sale_price'].sum().round(-4)

df_total["sum"] = [sum_mg, sum_sp]

df_total

,count,mean,std,min,25%,50%,75%,max,sum
monthly_gain,2927739.00,32770000.00,552030000.00,5.06,4310000.00,11580000.00,27580000.00,181237340000.00,95953120370000.00
average_sale_price,2927739.00,40000.00,180000.00,1.00,10000.00,20000.00,40000.00,165260000.00,111459210000.00


In [21]:
df_total.to_csv('df_total.csv', float_format = '%.2f', encoding = 'utf-8-sig')

### shop_name + shop_type_big + shop_type_small : df_nlp

In [128]:
# NLP용 preprocessing 
# shop_name, shop_type_big, shop_type_small = concat_text 
df_nlp1 = processed_data.copy()
df_nlp1['concat_text'] = df_nlp1['shop_name'] + ' ' + df_nlp1['shop_type_big'] + ' ' + df_nlp1['shop_type_small'] + ' '
df_nlp1.head()

,date,shop_code,shop_name,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,address1,address2,concat_text
0,201709,9381000145,고기.원칙,한식,고기구이/육회 전문점,127.12,37.49,19439629.62,31288,송파구,가락본동,고기.원칙 한식 고기구이/육회 전문점
1,201709,18211290214,동달식당 강남본점,한식,고기구이/육회 전문점,127.02,37.50,184992017.85,62762,서초구,서초2동,동달식당 강남본점 한식 고기구이/육회 전문점
2,201708,14946450207,서귀포혹돈,한식,고기구이/육회 전문점,126.90,37.47,217826541.22,89076,금천구,독산1동,서귀포혹돈 한식 고기구이/육회 전문점
3,201709,6188500699,(주)아이언미트 숙달돼지 사당점,한식,고기구이/육회 전문점,126.98,37.48,52677315.50,87160,서초구,방배2동,(주)아이언미트 숙달돼지 사당점 한식 고기구이/육회 전문점
4,201708,5598500495,민소한우(서린점),한식,고기구이/육회 전문점,126.98,37.57,75889137.73,93559,종로구,종로1.2.3.4가동,민소한우(서린점) 한식 고기구이/육회 전문점


In [129]:
df_nlp2 = df_nlp1[['concat_text', 'address1']]
df_nlp2.head()

,concat_text,address1
0,고기.원칙 한식 고기구이/육회 전문점,송파구
1,동달식당 강남본점 한식 고기구이/육회 전문점,서초구
2,서귀포혹돈 한식 고기구이/육회 전문점,금천구
3,(주)아이언미트 숙달돼지 사당점 한식 고기구이/육회 전문점,서초구
4,민소한우(서린점) 한식 고기구이/육회 전문점,종로구


In [130]:
from keras.preprocessing.text import Tokenizer

df_nlp = pd.DataFrame()
alist = df_nlp2['address1'].unique()
for add in alist : 
    tk = Tokenizer()
    tk.fit_on_texts(df_nlp2[df_nlp2['address1'] == add]['concat_text'])
    
    wordlist = list(tk.word_index.keys())[:30]
    df_word = pd.DataFrame(wordlist)
    df_nlp = pd.concat([df_nlp, df_word], axis=1)
    
    print(add, ": ", list(tk.word_index.items())[:30])
    print("\nvocab words 개수 : ", len(tk.word_index.items()))

송파구 :  [('전문점', 1), ('미분류', 2), ('음식', 3), ('카페', 4), ('한식', 5), ('소매', 6), ('분식', 7), ('커피숍', 8), ('다방', 9), ('디저트', 10), ('유흥주점', 11), ('별식', 12), ('퓨전요리', 13), ('수산물', 14), ('편의점', 15), ('식료품', 16), ('일반', 17), ('고기구이', 18), ('육회', 19), ('패스트푸드', 20), ('치킨', 21), ('닭강정', 22), ('주', 23), ('제과제빵떡케익', 24), ('백반', 25), ('한정식', 26), ('호프', 27), ('한식국물요리', 28), ('맥주', 29), ('일식', 30)]

vocab words 개수 :  10264
서초구 :  [('전문점', 1), ('미분류', 2), ('음식', 3), ('카페', 4), ('한식', 5), ('커피숍', 6), ('다방', 7), ('디저트', 8), ('소매', 9), ('분식', 10), ('수산물', 11), ('유흥주점', 12), ('편의점', 13), ('일반', 14), ('주', 15), ('별식', 16), ('퓨전요리', 17), ('고기구이', 18), ('육회', 19), ('패스트푸드', 20), ('한정식', 21), ('백반', 22), ('한식국물요리', 23), ('중식', 24), ('식료품', 25), ('제과제빵떡케익', 26), ('치킨', 27), ('일식', 28), ('닭강정', 29), ('중식당', 30)]

vocab words 개수 :  8679
금천구 :  [('전문점', 1), ('미분류', 2), ('음식', 3), ('한식', 4), ('카페', 5), ('소매', 6), ('분식', 7), ('다방', 8), ('커피숍', 9), ('디저트', 10), ('유흥주점', 11), ('수산물', 12), ('편의점', 13), ('식료품', 14), ('별식

은평구 :  [('전문점', 1), ('미분류', 2), ('음식', 3), ('한식', 4), ('카페', 5), ('소매', 6), ('분식', 7), ('커피숍', 8), ('유흥주점', 9), ('다방', 10), ('별식', 11), ('퓨전요리', 12), ('디저트', 13), ('수산물', 14), ('편의점', 15), ('일반', 16), ('닭강정', 17), ('치킨', 18), ('고기구이', 19), ('육회', 20), ('패스트푸드', 21), ('식료품', 22), ('호프', 23), ('맥주', 24), ('제과제빵떡케익', 25), ('백반', 26), ('한정식', 27), ('오리', 28), ('닭요리', 29), ('중식', 30)]

vocab words 개수 :  5911
성북구 :  [('전문점', 1), ('미분류', 2), ('음식', 3), ('카페', 4), ('한식', 5), ('소매', 6), ('분식', 7), ('다방', 8), ('커피숍', 9), ('디저트', 10), ('유흥주점', 11), ('별식', 12), ('퓨전요리', 13), ('수산물', 14), ('편의점', 15), ('치킨', 16), ('닭강정', 17), ('일반', 18), ('패스트푸드', 19), ('고기구이', 20), ('육회', 21), ('제과제빵떡케익', 22), ('식료품', 23), ('백반', 24), ('한정식', 25), ('호프', 26), ('맥주', 27), ('한식국물요리', 28), ('중식', 29), ('오리', 30)]

vocab words 개수 :  5611
강동구 :  [('전문점', 1), ('미분류', 2), ('음식', 3), ('한식', 4), ('카페', 5), ('소매', 6), ('분식', 7), ('커피숍', 8), ('다방', 9), ('유흥주점', 10), ('디저트', 11), ('수산물', 12), ('별식', 13), ('퓨전요리', 14), ('편의점',

In [131]:
df_nlp.columns = alist 
df_nlp

,송파구,서초구,금천구,종로구,동대문구,중구,구로구,도봉구,중랑구,광진구,...,영등포구,강남구,노원구,용산구,은평구,성북구,강동구,서대문구,과천시,광명시
0,전문점,전문점,전문점,전문점,전문점,전문점,전문점,전문점,전문점,전문점,...,전문점,전문점,전문점,전문점,전문점,전문점,전문점,전문점,전문점,부흥축산
1,미분류,미분류,미분류,미분류,미분류,음식,미분류,미분류,미분류,미분류,...,미분류,미분류,미분류,미분류,미분류,미분류,미분류,미분류,bbq,소매
2,음식,음식,음식,음식,음식,미분류,음식,음식,음식,음식,...,음식,음식,음식,음식,음식,음식,음식,음식,치킨앤비어,식료품
3,카페,카페,한식,카페,한식,카페,한식,한식,한식,카페,...,카페,카페,카페,카페,한식,카페,한식,카페,별식,음식
4,한식,한식,카페,한식,카페,한식,카페,카페,카페,한식,...,한식,한식,한식,한식,카페,한식,카페,한식,퓨전요리,미분류
5,소매,커피숍,소매,커피숍,소매,소매,소매,소매,소매,분식,...,소매,다방,소매,다방,소매,소매,소매,다방,치킨,먹빵
6,분식,다방,분식,다방,분식,다방,분식,분식,분식,유흥주점,...,다방,커피숍,분식,커피숍,분식,분식,분식,분식,닭강정,NaN
7,커피숍,디저트,다방,디저트,커피숍,커피숍,유흥주점,커피숍,유흥주점,다방,...,커피숍,디저트,다방,디저트,커피숍,다방,커피숍,커피숍,미가식당,NaN
8,다방,소매,커피숍,분식,다방,디저트,다방,유흥주점,수산물,커피숍,...,분식,소매,커피숍,분식,유흥주점,커피숍,다방,소매,한식,NaN
9,디저트,분식,디저트,유흥주점,디저트,분식,커피숍,다방,별식,디저트,...,디저트,분식,디저트,유흥주점,다방,디저트,유흥주점,디저트,한정식,NaN


In [132]:
df_nlp.to_csv('df_nlp.csv',encoding = 'utf-8-sig')

### shop_name

In [123]:
# NLP용 preprocessing 
# shop_name, shop_type_big, shop_type_small = concat_text 
df_nlp1 = processed_data.copy()
df_nlp1.head()

,date,shop_code,shop_name,shop_type_big,shop_type_small,longitude,latitude,monthly_gain,average_sale_price,address1,address2,concat_text
0,201709,9381000145,고기.원칙,한식,고기구이/육회 전문점,127.12,37.49,19439629.62,31288,송파구,가락본동,고기.원칙
1,201709,18211290214,동달식당 강남본점,한식,고기구이/육회 전문점,127.02,37.50,184992017.85,62762,서초구,서초2동,동달식당 강남본점
2,201708,14946450207,서귀포혹돈,한식,고기구이/육회 전문점,126.90,37.47,217826541.22,89076,금천구,독산1동,서귀포혹돈
3,201709,6188500699,(주)아이언미트 숙달돼지 사당점,한식,고기구이/육회 전문점,126.98,37.48,52677315.50,87160,서초구,방배2동,(주)아이언미트 숙달돼지 사당점
4,201708,5598500495,민소한우(서린점),한식,고기구이/육회 전문점,126.98,37.57,75889137.73,93559,종로구,종로1.2.3.4가동,민소한우(서린점)


In [124]:
from keras.preprocessing.text import Tokenizer

df_nlp = pd.DataFrame()
alist = df_nlp1['address1'].unique()
for add in alist : 
    tk = Tokenizer()
    tk.fit_on_texts(df_nlp1[df_nlp1['address1'] == add]['shop_name'])
    
    wordlist = list(tk.word_index.keys())[:30]
    df_word = pd.DataFrame(wordlist)
    df_nlp = pd.concat([df_nlp, df_word], axis=1)
    
    print(add, ": ", list(tk.word_index.items())[:30])
    print("\nvocab words 개수 : ", len(tk.word_index.items()))

송파구 :  [('주', 1), ('주식회사', 2), ('gs25', 3), ('잠실점', 4), ('문정점', 5), ('카페', 6), ('유', 7), ('방이점', 8), ('코리아세븐', 9), ('세븐일레븐', 10), ('씨유', 11), ('송파점', 12), ('cu', 13), ('유한회사', 14), ('가락점', 15), ('cafe', 16), ('신천점', 17), ('파리바게뜨', 18), ('coffee', 19), ('커피', 20), ('석촌호수점', 21), ('이마트24', 22), ('문정법조타운점', 23), ('김밥', 24), ('미니스톱', 25), ('송파', 26), ('위례점', 27), ('거여점', 28), ('이디야', 29), ('투다리', 30)]

vocab words 개수 :  10210
서초구 :  [('주', 1), ('주식회사', 2), ('서초점', 3), ('gs25', 4), ('교대점', 5), ('양재점', 6), ('코리아세븐', 7), ('방배점', 8), ('카페', 9), ('씨유', 10), ('cafe', 11), ('세븐일레븐', 12), ('강남역점', 13), ('교대역점', 14), ('강남점', 15), ('파리바게뜨', 16), ('남부터미널점', 17), ('반포점', 18), ('커피', 19), ('coffee', 20), ('이마트24', 21), ('방배역점', 22), ('커피빈코리아', 23), ('잠원점', 24), ('cu', 25), ('미니스톱', 26), ('더', 27), ('김밥천국', 28), ('서래마을점', 29), ('서초', 30)]

vocab words 개수 :  8612
금천구 :  [('주', 1), ('gs25', 2), ('독산점', 3), ('가산점', 4), ('가산디지털점', 5), ('세븐일레븐', 6), ('주식회사', 7), ('씨유', 8), ('금천점', 9), ('cu', 10), ('미니스톱', 11

노원구 :  [('주', 1), ('gs25', 2), ('노원점', 3), ('공릉점', 4), ('주식회사', 5), ('중계점', 6), ('노원역점', 7), ('상계점', 8), ('씨유', 9), ('파리바게뜨', 10), ('수락산역점', 11), ('세븐일레븐', 12), ('cu', 13), ('코리아세븐', 14), ('광운대점', 15), ('미니스톱', 16), ('이디야', 17), ('배스킨라빈스', 18), ('이마트24', 19), ('본죽', 20), ('수락산점', 21), ('화랑대역점', 22), ('쥬씨', 23), ('요거프레소', 24), ('상계역점', 25), ('마들역점', 26), ('석계역점', 27), ('치킨', 28), ('공릉역점', 29), ('은행사거리점', 30)]

vocab words 개수 :  5397
용산구 :  [('주', 1), ('주식회사', 2), ('gs25', 3), ('이태원점', 4), ('코리아세븐', 5), ('용산점', 6), ('숙대점', 7), ('한남점', 8), ('용산아이파크몰점', 9), ('씨유', 10), ('동부이촌점', 11), ('이마트24', 12), ('세븐일레븐', 13), ('카페', 14), ('cu', 15), ('파리바게뜨', 16), ('김밥천국', 17), ('서울역점', 18), ('보광점', 19), ('이태원', 20), ('투썸플레이스', 21), ('이촌점', 22), ('cafe', 23), ('신용산점', 24), ('숙대입구점', 25), ('후암점', 26), ('coffee', 27), ('용산', 28), ('남영점', 29), ('빽다방', 30)]

vocab words 개수 :  5658
은평구 :  [('gs25', 1), ('주', 2), ('연신내점', 3), ('씨유', 4), ('불광점', 5), ('세븐일레븐', 6), ('응암점', 7), ('주식회사', 8), ('코리아세븐', 9), ('은평뉴타운

In [125]:
df_nlp.columns = alist 
df_nlp

,송파구,서초구,금천구,종로구,동대문구,중구,구로구,도봉구,중랑구,광진구,...,영등포구,강남구,노원구,용산구,은평구,성북구,강동구,서대문구,과천시,광명시
0,주,주,주,주,주,주,주,창동점,gs25,주,...,주,주,주,주,gs25,주,주,주,bbq,부흥축산
1,주식회사,주식회사,gs25,대학로점,gs25,주식회사,구로점,주,주,건대점,...,여의도점,주식회사,gs25,주식회사,주,성신여대점,천호점,신촌점,치킨앤비어,먹빵
2,gs25,서초점,독산점,주식회사,장안점,gs25,gs25,gs25,신내점,gs25,...,gs25,gs25,노원점,gs25,연신내점,gs25,gs25,주식회사,미가식당,NaN
3,잠실점,gs25,가산점,광화문점,경희대점,명동점,구로디지털점,쌍문점,면목점,주식회사,...,주식회사,역삼점,공릉점,이태원점,씨유,정릉점,길동점,gs25,정성한끼,NaN
4,문정점,교대점,가산디지털점,카페,코리아세븐,세븐일레븐,신도림점,세븐일레븐,상봉점,구의점,...,영등포점,코리아세븐,주식회사,코리아세븐,불광점,주식회사,주식회사,이대점,NaN,NaN
5,카페,양재점,세븐일레븐,종로점,세븐일레븐,카페,주식회사,주식회사,씨유,씨유,...,씨유,대치점,중계점,용산점,세븐일레븐,종암점,명일점,홍제점,NaN,NaN
6,유,코리아세븐,주식회사,gs25,cu,코리아세븐,고척점,방학점,세븐일레븐,세븐일레븐,...,문래점,카페,노원역점,숙대점,응암점,세븐일레븐,씨유,카페,NaN,NaN
7,방이점,방배점,씨유,광화문,씨유,씨유,개봉점,cu,묵동점,cu,...,당산점,강남점,상계점,한남점,주식회사,파리바게뜨,암사점,연희점,NaN,NaN
8,코리아세븐,카페,금천점,커피,주식회사,신당점,씨유,씨유,망우점,중곡점,...,코리아세븐,씨유,씨유,용산아이파크몰점,코리아세븐,씨유,코리아세븐,명지대점,NaN,NaN
9,세븐일레븐,씨유,cu,종각점,외대점,cu,cu,쌍문역점,코리아세븐,자양점,...,coffee,삼성점,파리바게뜨,씨유,은평뉴타운점,코리아세븐,세븐일레븐,씨유,NaN,NaN


In [127]:
df_nlp.to_csv('df_shopname.csv',encoding = 'utf-8-sig')